In [2]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:14 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:15 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

In [3]:
!wget https://jdbc.postgresql.org/download/postgresql-42.3.3.jar

--2022-03-07 08:05:58--  https://jdbc.postgresql.org/download/postgresql-42.3.3.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1039047 (1015K) [application/java-archive]
Saving to: ‘postgresql-42.3.3.jar’

postgresql-42.3.3.j 100%[===================>]   1015K  6.36MB/s    in 0.2s    

2022-03-07 08:05:59 (6.36 MB/s) - ‘postgresql-42.3.3.jar’ saved [1039047/1039047]



In [4]:
 # Start Spark session
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("WRDB").config("spark.driver.extraClassPath","/content/postgresql-42.3.3.jar").getOrCreate()

In [5]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url ="https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Camera_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

In [6]:
df = spark.read.option("header", "true").csv(SparkFiles.get("amazon_reviews_us_Camera_v1_00.tsv.gz"), inferSchema=True, sep="\t")
# Show DataFrame
df.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    2975964|R1NBG94582SJE2|B00I01JQJM|     860486164|GoPro Rechargeabl...|          Camera|          5|            0|          0|   N|                Y|          Five Stars|                  ok| 2015-08-31|
|         US|   23526356|R273DCA6Y0H9V7|B00TCO0ZAA|     292641483|Professional 58mm...|          Camera|          5|    

In [7]:
number_of_rows = df.count()
print("The number of rows are:", number_of_rows)

The number of rows are: 1801974


In [8]:
review_id_table = df["review_id", "customer_id", "product_id", "product_parent", "review_date"]
review_id_table.show(5)
review_id_table.dtypes

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R1NBG94582SJE2|    2975964|B00I01JQJM|     860486164| 2015-08-31|
|R273DCA6Y0H9V7|   23526356|B00TCO0ZAA|     292641483| 2015-08-31|
| RQVOXO7WUOFK6|   52764145|B00B7733E0|      75825744| 2015-08-31|
|R1KWKSF21PO6HO|   47348933|B006ZN4U34|     789352955| 2015-08-31|
|R38H3UO1J190GI|   33680700|B00HUEBGMU|      19067902| 2015-08-31|
+--------------+-----------+----------+--------------+-----------+
only showing top 5 rows



[('review_id', 'string'),
 ('customer_id', 'int'),
 ('product_id', 'string'),
 ('product_parent', 'int'),
 ('review_date', 'string')]

In [9]:
products_table = df["product_id", "product_title"]
products_table.show(5)
products_table.dtypes

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00I01JQJM|GoPro Rechargeabl...|
|B00TCO0ZAA|Professional 58mm...|
|B00B7733E0|Spy Tec Z12 Motio...|
|B006ZN4U34|Celestron UpClose...|
|B00HUEBGMU|Vidpro XM-L Wired...|
+----------+--------------------+
only showing top 5 rows



[('product_id', 'string'), ('product_title', 'string')]

In [10]:
from pyspark.sql.functions import count
customers_table = df.groupBy("customer_id").agg(count("customer_id").alias("customer_count"))
customers_table.show(5)
customers_table.dtypes

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   52695798|             1|
|   48363612|             5|
|   46909180|             1|
|   45595220|             1|
|   50372387|             2|
+-----------+--------------+
only showing top 5 rows



[('customer_id', 'int'), ('customer_count', 'bigint')]

In [11]:
vine_table = df["review_id", "star_rating", "helpful_votes", "total_votes", "vine"]
vine_table.show(5)
vine_table.dtypes

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R1NBG94582SJE2|          5|            0|          0|   N|
|R273DCA6Y0H9V7|          5|            0|          0|   N|
| RQVOXO7WUOFK6|          2|            1|          1|   N|
|R1KWKSF21PO6HO|          5|            0|          0|   N|
|R38H3UO1J190GI|          5|            1|          1|   N|
+--------------+-----------+-------------+-----------+----+
only showing top 5 rows



[('review_id', 'string'),
 ('star_rating', 'int'),
 ('helpful_votes', 'int'),
 ('total_votes', 'int'),
 ('vine', 'string')]

In [ ]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://localhost:5432/awrdb"
config={"user":"postgres", 
          "password": "******", 
          "driver":"org.postgresql.Driver"}

In [ ]:
review_id_table.write.jdbc(url=jdbc_url,table='review_id_table',mode=mode,properties=config)

In [ ]:
products_table.write.jdbc(url=jdbc_url,table='products',mode=mode,properties=config)

In [ ]:
customers_table.write.jdbc(url=jdbc_url,table='customers',mode=mode,properties=config)

In [ ]:
vine_table.write.jdbc(url=jdbc_url,table='vine_table',mode=mode,properties=config)